In [2]:
import pandas as pd
import math

In [3]:
df=pd.read_csv("quotes.csv")

In [4]:
df.head()

,quotetext,character,title,author,year
0,Maybe 'okay' will be our 'always.,NaN,'The Fault in Our Stars','John Green',2012
1,"'I wonder what Piglet is doing,' thought Pooh....",NaN,Winnie-the-Pooh,A.A. Milne,1926
2,"Anarchy means 'without leaders', not 'without ...",V,V for Vendetta,Alan Moore,1982
3,Everybody is special. Everybody. Everybody is ...,V,V for Vendetta,Alan Moore,1982
4,"Happiness is a prison, Evey. Happiness is the ...",V,V for Vendetta,Alan Moore,1982
